In [1]:
import bs4
from collections import defaultdict    
from bs4 import BeautifulSoup    
import requests
import pandas as pd
import pymongo
import json
from bson import json_util


In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Create a new database
db = client.clinical_trial_db

# Create a new collection and reset the collection
collection = db.ct_gov
db.drop_collection(collection)

{'ns': 'clinical_trial_db.ct_gov', 'nIndexesWas': 1, 'ok': 1.0}

In [3]:
def clinicalTrialsGov(nctid):
    data = defaultdict(list)
    soup = BeautifulSoup(requests.get("https://clinicaltrials.gov/ct2/show/" + nctid + "?displayxml=true").text, "xml")
    subset = ['name','status','city', 'zip']
    for tag in soup.find_all(subset):
        data['ct{}'.format(tag.name.capitalize())].append(tag.get_text(strip=True))

    for key in data:
        print('{}: {}'.format(key, ', '.join(data[key])))
    return data

In [4]:
data = clinicalTrialsGov('NCT01592370')

ctName: Local Institution, Local Institution, Local Institution, Division Of Hematology & Oncology Ctr. For Health Sciences, University of Colorado Denver, Yale University School Of Medicine, Local Institution, Local Institution, Cancer Institute Of Florida, Local Institution, Local Institution, Local Institution, Indiana University Melvin & Bren Simon Cancer Center, Local Institution, University Of Kansas Medical Center, The Sidney Kimmel Comprehensive Cancer Center, Dana Farber Cancer Institute, Dana-Farber Cancer Institute, University Of Michigan Health System, Mayo Clinic, Local Institution, Nebraska Cancer Specialists, John Theurer Cancer Center, Memorial Sloan Kettering Cancer Center, Local Institution, Ohio State University, Local Institution, OHSU Center for Hematologic Malignancies, Abramson Cancer Center, Fox Chase Cancer Center, Local Institution, Local Institution, Local Institution, Local Institution, Huntsman Cancer Institute At The Univ. Of Utah, Local Institution, Local

In [5]:
df = pd.DataFrame({'name':data['ctName'],'status':data['ctStatus'],'city':data['ctCity'],'zip':data['ctZip']})
df.insert(0, 'NCT', 'NCT01592370')
# df.reset_index().to_json(orient='records')

# ctjson = df.to_json (r'C:\Users\krystal.sanhudo\Desktop\ETLct\json.json1', orient='records')

records = json.loads(df.T.to_json()).values()
db.ct_gov.insert_many(records)

records=db.ct_gov.find()
for record in records:
    print(record)
# collection.insert_one(json)

In [9]:
records=db.ct_gov.find()
for record in records:
    print(record)

{'_id': ObjectId('5cb205efc1a0790700526ef4'), 'NCT': 'NCT01592370', 'name': 'Local Institution', 'status': 'Withdrawn', 'city': 'Little Rock', 'zip': '72205'}
{'_id': ObjectId('5cb205efc1a0790700526ef5'), 'NCT': 'NCT01592370', 'name': 'Local Institution', 'status': 'Not yet recruiting', 'city': 'Fresno', 'zip': '93701'}
{'_id': ObjectId('5cb205efc1a0790700526ef6'), 'NCT': 'NCT01592370', 'name': 'Local Institution', 'status': 'Withdrawn', 'city': 'Long Beach', 'zip': '90813'}
{'_id': ObjectId('5cb205efc1a0790700526ef7'), 'NCT': 'NCT01592370', 'name': 'Division Of Hematology & Oncology Ctr. For Health Sciences', 'status': 'Active, not recruiting', 'city': 'Los Angeles', 'zip': '90095'}
{'_id': ObjectId('5cb205efc1a0790700526ef8'), 'NCT': 'NCT01592370', 'name': 'University of Colorado Denver', 'status': 'Recruiting', 'city': 'Aurora', 'zip': '80045'}
{'_id': ObjectId('5cb205efc1a0790700526ef9'), 'NCT': 'NCT01592370', 'name': 'Yale University School Of Medicine', 'status': 'Completed', 'ci

In [7]:
# export_csv = df.to_csv (r'C:\Users\krystal.sanhudo\Desktop\ETLct\ct-ETL.csv', index = None, header=True)

In [8]:
# df = pd.DataFrame({'name':data['ctName'],'status':data['ctStatus'],'address':data['ctAddress']})